In [ ]:
#Importing all the required libraries
import numpy as np
from keras.models import load_model
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import keras
import cv2
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers , models, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50V2
train_dir = '../input/fer2013/train/'
test_dir = '../input/fer2013/test/'



In [ ]:
#This is for the mobile net
mobileNet = load_model('../input/savedmodels/mobilenet_model.h5')
test_datagen = ImageDataGenerator(rescale=1./255)

#Importing the testing dataset
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=False,

                                                color_mode='grayscale',
                                                class_mode='categorical')

#We are now predicting the results using predict
mp = mobileNet.predict(test_set)
mobileNetPredictions = np.argmax(mp, axis=-1)


In [ ]:

#Importing the googleNet model
googleNet = load_model('../input/savedmodels/googlenet_model.h5')
test_datagen = ImageDataGenerator(rescale=1./255)

#Now we are importing the test dataset
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=False,
                                                color_mode='grayscale',
                                                class_mode='categorical')
#We are now predicting using the googleNet dataset
mp = googleNet.predict(test_set)
googleNetPredictions = np.argmax(mp, axis=-1)


In [ ]:
#This is for the resNet dataset
resNet = load_model('../input/savedmodels/resnet_model.h5')
test_datagen = ImageDataGenerator(rescale=1./255)
#We are importing the data values from the directory and
test_set = test_datagen.flow_from_directory(test_dir,
                                                class_mode="categorical",
    target_size=(224,224),
    color_mode="rgb",
    shuffle=False,
    batch_size=64,)

#Now we are predicting the values and then storing them
mp = resNet.predict(test_set)
resNetPredictions = np.argmax(mp, axis=-1)


In [ ]:
#Loading the model
inceptionNet = load_model('../input/savedmodels/inceptionnet_model.h5')
test_datagen = ImageDataGenerator(rescale=1./255)

#Importing testing dataset from the directory
test_set = test_datagen.flow_from_directory(test_dir,target_size = (250,188),
                                                color_mode = 'rgb',
                                             batch_size = 64,
                                             class_mode = 'categorical',
                                             shuffle = False)

#Now, we are predicting the values using the inception net model
mp = inceptionNet.predict(test_set)
incpetionNetPredictions = np.argmax(mp, axis=-1)



In [ ]:
#Just checking the structure of inceptionNet
inceptionNet = load_model('../input/savedmodels/inceptionnet_model.h5')
inceptionNet.summary()

In [ ]:
#Loading the LSTM model
lstm = load_model('../input/savedmodels/lstm_model.h5')

test_datagen = ImageDataGenerator(rescale=1./255)
#Importing the test dataset from the directory
test_set = test_datagen.flow_from_directory(test_dir,target_size = (48,48),
                                                color_mode = 'grayscale',
                                             batch_size = 64,
                                             class_mode = 'categorical',
                                            
                                             shuffle = False)

#Now, we are predicting the values using lstm and then storing them
mp = lstm.predict(test_set)
lstmPredictions = np.argmax(mp, axis=-1)






In [ ]:
#This is for the baseline model that we had trained and tehn predicting the test values

baseline = load_model('../input/savedmodels/baseline_model.h5')
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,target_size = (48,48),
                                                color_mode = 'grayscale',
                                             batch_size = 64,
                                             class_mode = 'categorical',
                                            
                                             shuffle = False)

mp = baseline.predict(test_set)
baselinePredictions = np.argmax(mp, axis=-1)



In [ ]:
#These are the predicted values, using all the imported models
predictionSet = [googleNetPredictions, resNetPredictions,incpetionNetPredictions, mobileNetPredictions,lstmPredictions,baselinePredictions]

ensemble_predictions = []
#We are taking the maximum voting technique
#We are taking the maximum voted class
for i in range(len(incpetionNetPredictions)):
    predictions = [predictionSet[0][i], predictionSet[1][i], predictionSet[2][i], predictionSet[3][i],predictionSet[4][i],predictionSet[5][i]]
    prediction = max(set(predictions), key=predictions.count)
    ensemble_predictions.append(prediction)
    
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_set.classes, ensemble_predictions)
print("Ensemble accuracy:", accuracy)

    

In [ ]:
#Printing out the accuracies
accuracy = accuracy_score(test_set.classes, predictionSet[0])
print("Google:", accuracy)
accuracy = accuracy_score(test_set.classes, predictionSet[4])
print("LSTM:", accuracy)
accuracy = accuracy_score(test_set.classes, predictionSet[1])
print("Resnet:", accuracy)
accuracy = accuracy_score(test_set.classes, predictionSet[2])
print("Inception:", accuracy)
accuracy = accuracy_score(test_set.classes, predictionSet[3])
print("Mobile:", accuracy)
accuracy = accuracy_score(test_set.classes, predictionSet[5])
print("Baseline:", accuracy)



In [ ]:
lstm = load_model('../input/savedmodels/lstm_model.h5')

In [ ]:
lstm.summary()

In [ ]:
test_dir2 = '../input/young-affectnet-hq/'
test_datagen = ImageDataGenerator(rescale=1./255)
#Importing the testing values
test_set_2 = test_datagen.flow_from_directory(test_dir2,target_size = (48,48),
                                                color_mode = 'grayscale',
                                             batch_size = 64,
                                             class_mode = 'categorical',
                                             shuffle = False)

In [ ]:
#These are the images and the labels
x_test_arr, y_test = next(test_set_2)

In [ ]:
#Converting the one hot encoded values to normal
y_test_nothot = np.argmax(y_test, axis = 1)
y_test_nothot.shape

In [ ]:
test_set_2.class_indices

In [ ]:
from tqdm import notebook
x_test_new = []
y_test_new = []

for i in notebook.tqdm(range(len(y_test_nothot))):
    if(y_test_nothot[i] != 1):
        x_test_new += [x_test_arr[i]]
        y_test_new += [y_test_nothot[i]]
        

In [ ]:
#Converting them to numpy values
x_test_new = np.array(x_test_new)
y_test_new = np.array(y_test_new)

In [ ]:
mp = lstm.predict(x_test_new)
lstmPredictions = np.argmax(mp, axis=1)

In [ ]:
lstmPredictions.shape

In [ ]:
#Accuracy score of lstm individually
from sklearn.metrics import accuracy_score

accuracy_score(y_test_new, lstmPredictions)